<a href="https://colab.research.google.com/github/Manasa-Sai/PricePrediction/blob/main/data_cleanup.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import matplotlib.pyplot as plt
import datetime as dt
# common imports
import zipfile
import time
# import datetime, timedelta
import datetime
from datetime import datetime, timedelta
from datetime import date
from dateutil import relativedelta
from io import StringIO
import pandas as pd
import pickle
from sklearn.base import BaseEstimator
from sklearn.base import TransformerMixin
from io import StringIO
import requests
import json
from sklearn.preprocessing import LabelEncoder, MinMaxScaler, StandardScaler
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
%matplotlib inline 
import os
import math
from subprocess import check_output
from IPython.display import display
import logging
import yaml
from collections import Counter
import re
import os
import numbers

In [ ]:
def get_config(config_file):
    ''' open config file with name config_file that contains parameters
    for this module and return Python object

    Args:
        config_file: filename containing config parameters

    Returns:
        config: Python dictionary with config parms from config file - dictionary


    '''
    current_path = os.getcwd()
    path_to_yaml = os.path.join(current_path, config_file)
    print("path_to_yaml " + path_to_yaml)
    try:
        with open(path_to_yaml, 'r') as c_file:
            config = yaml.safe_load(c_file)
        return config
    except Exception as error:
        print('Error reading the config file ' + str(error))

In [ ]:
def print_config_values(config):
    for val in config:
        print("config value ",val," ",str(config[val]))

In [ ]:

def get_path():
    ''' get the path for data files

    Returns:
        path: path for data directory

    '''
    rawpath = os.getcwd()
    # data is in a directory called "data" that is a sibling to the directory
    # containing the notebook
    path = os.path.abspath(os.path.join(rawpath, '..', 'data'))
    return path
    

In [ ]:
def ingest_data(path,input_csv,pickled_input_dataframe,save_raw_dataframe,load_from_scratch):
    ''' load data into dataframe
    Args:
        path: path containing input file
        input_csv: input file name
        pickled_input_dataframe: pickled version of input file

    Returns:
        path: path for data directory
    '''
    if load_from_scratch:
        unpickled_df = pd.read_csv(os.path.join(path,input_csv)) 
        if save_raw_dataframe:
            file_name = os.path.join(path,pickled_input_dataframe)
            print("file_name is ",file_name)
            unpickled_df.to_pickle(file_name)
    else:
        unpickled_df = pd.read_pickle(os.path.join(path,pickled_input_dataframe))
        logging.debug("reloader done")
    return(unpickled_df)

In [ ]:
def fill_missing(dataset,columns,defaults):
    ''' replace missing values with placeholders by column type
    
    Args:
        dataset: dataframe in which missing values being processed
        columns: dictionary of columns with keys that are column types and values that are the column names of that type
        defaults: dictionary of replacement values for missing values by column type

    Returns:
        dataset: dataframe with missing values replaced with default values

    '''
    logging.debug("before mv")
    for column_category in columns:
        print("column_category is "+str(column_category))
        for col in columns[column_category]:
            print("filling mising values in col "+str(col)+" with default "+str(defaults[column_category]))
            dataset[col].fillna(defaults[column_category],inplace = True)
            print("in mv Missing values in ",col," ",str(dataset[col].isna().sum()))
     
    return(dataset)


In [ ]:
def not_in_list(x, list):
    ''' check if a value is in a list
    Args:
        x: value to check
        list: list in which to check for the value

    Returns:
        retur_val: 1 if value is in not in list, 0 otherwise
    '''
    if x in list:
        return_val = 0
    else:
        return_val = 1
    return(return_val)

In [ ]:
def neg_val(x):
    ''' check if a value is in a list
    Args:
        x: value to check
    
    Returns:
        retur_val: 1 if value is negative, 0 otherwise
    '''
    if x >= 0:
        return_val = 0
    else:
        return_val = 1
    return(return_val)

In [ ]:

def basic_assessment(df,columns,valid_values,non_neg_continuous):
    ''' assess the values in a dataframe
    Args:
        df: dataframe for assessment
        columns: dictionary of column names by category
        valid_values: dictionary of valid values for categorical columns with limited number of valid values
        non_neg_continuous: list of continuous columns with only non-negative values as valid
    '''
    for col in list(df):
        print("Missing values in ",col," ",str(df[col].isna().sum()))
        print("Distinct values in ",col," ",str(df[col].nunique()))
    # for categorical columns with a limited number of valid values, count the number of invalid values by column
    for col in valid_values:
        print("non-valid values in column ",col," ",str(df[col].apply(lambda x:not_in_list(x,valid_values[col])).sum()))
    # count non-numeric values in continuous columns
    for col in columns['continuous']:
        # mask = pd.to_numeric(df['Hours_Worked'], errors='coerce').isna()
        mask = pd.to_numeric(df[col], errors='coerce').isna()
        print("non-numeric values in continuous col ",col," ",str(mask.sum()))
        # if there are no non-numeric values in the column and it muast have non-negative values, count negative values
        if (mask.sum()==0) and (col in non_neg_continuous):
            print("negative values in colum ",col," ",str(df[col].apply(lambda x:neg_val(x)).sum()))

In [ ]:

def out_of_range(x,max,min):
    ''' count whether a value is in a range
    Args:
        x: value to check in range
        max: top of the range to check
        min: bottom of the range to check
        
    Returns:
        ret_val: 1 if out of range, 0 otherwise
    '''
    if x > max or x < min:
        return_val = 1
    else:
        return_val = 0
    return(return_val)


In [ ]:

def out_of_bounding_box(latitude,longitude,bounding_box):
    ''' count whether a location is within a bounding box
    Args:
        latitude: latitude portion of location
        longitude: longitude portion of location
        bounding_box: dictionary with max and min values to compare the location with
        min: bottom of the range to check
        
    Returns:
        ret_val: 1 if out of range, 0 otherwise
    '''    
    if ((latitude <= bounding_box['max_lat']) and (latitude >= bounding_box['min_lat'])) \
    and ((longitude <= bounding_box['max_long']) and (longitude >= bounding_box['min_long'])):
         ret_val = 0
    else:
         ret_val = 1
    return(ret_val)

In [ ]:
def geo_assessment(df,bounding_box):
    ''' assess the geo columns in a dataframe by counting how many latitude and longitude values are outside the bounding box
    Args:
        df: dataframe for assessment
        bounding_box: dictionary of maximum and minimum valid latitude and longitude values
    ''' 
    # count the number of entries in the latitude column that are above or below a given amount
    # df['col_3'] = df.apply(lambda x: get_sublist(x.col_1, x.col_2), axis=1)
    print("latitude out of bounds count ",str(df['latitude'].apply(lambda x:out_of_range(x,bounding_box['max_lat'],bounding_box['min_lat'])).sum()))
    print("longitude out of bounds count ",str(df['longitude'].apply(lambda x:out_of_range(x,bounding_box['max_long'],bounding_box['min_long'])).sum()))
    print("location out of bounds count ",str(df.apply(lambda x: out_of_bounding_box(x.latitude,x.longitude,bounding_box), axis=1).sum()))

In [ ]:

def replace_invalid_values(df,columns,valid_values,invalid_value_replacements,non_neg_continuous):
    ''' replace invalid with placeholders
    Args:
        df: dataframe for assessment
        columns: dictionary of column names by category
        valid_values: dictionary of valid values for categorical columns with limited number of valid values
        invalid_value_replacements: dictionary of replacement values by column category
        non_neg_continuous: list of continuous columns with only non-negative values as valid
    Returns:
        df: updated dataframe
    '''
    # for categorical columns with a limited number of valid values, replace invalid values with placeholder
    for col in valid_values:
        print("non-valid values in column ",col," ",str(df[col].apply(lambda x:not_in_list(x,valid_values[col])).sum()))
        df[col] = df[col].apply(lambda x:replace_if_not_in_list(x,invalid_value_replacements['categorical'],valid_values[col]))
    # for continuous columns, replace non-numeric values and, where not valid, negative values
    for col in columns['continuous']:
        # replace non-numeric values
        df[col] = df[col].apply(lambda x:replace_if_non_numeric(x,invalid_value_replacements['continuous']))
        # for continuous columns where valid values must be non-negative, replace negative values
        if col in non_neg_continuous:
            df[col] = df[col].apply(lambda x:replace_if_neg(x,invalid_value_replacements['continuous'])) 
    return(df)

In [ ]:
def replace_if_non_numeric(x, replace_x):
    ''' check if a value is non-numeric and replace if so
    Args:
        x: value to check
    
    Returns:
        retur_val: 1 if value is negative, 0 otherwise
    '''
    if isinstance(x, numbers.Number):
        return_val = x
    else:
        return_val = replace_x
    return(return_val)

In [ ]:
def replace_if_not_in_list(x, replace_x, list):
    ''' check if a value is in a list
    Args:
        x: value to check
        list: list in which to check for the value

    Returns:
        retur_val: 1 if value is in not in list, 0 otherwise
    '''
    if x in list:
        return_val = x
    else:
        return_val = replace_x
    return(return_val)

In [ ]:
def replace_if_neg(x, replace_x):
    ''' check if a value is negative and replace if so
    Args:
        x: value to check
    
    Returns:
        retur_val: 1 if value is negative, 0 otherwise
    '''
    if x >= 0:
        return_val = x
    else:
        return_val = replace_x
    return(return_val)

In [ ]:
def geo_replacement(df,replace_lat, replace_long, bounding_box):
    ''' assess the geo columns in a dataframe by counting how many latitude and longitude values are outside the bounding box
    Args:
        df: dataframe for assessment
        replace_lat: replacement for latitude if out of bounds
        replace_long: replacement for longitude if out of bounds
        bounding_box: dictionary of maximum and minimum valid latitude and longitude values
    Returns:
        df: updated dataframe
    ''' 
    # df['latitude','longitude'] = df['latitude','longitude'].apply(lambda x,y:replace_if_outside_bounding_box(x,y,replace_lat,replace_long,bounding_box))
    df['latitude','longitude'] = df.apply(lambda x:replace_if_outside_bounding_box(x.latitude,x.longitude,replace_lat,replace_long,bounding_box),axis=1)
    return(df)

In [ ]:

def replace_if_outside_bounding_box(latitude,longitude, replace_lat, replace_long, bounding_box):
    ''' count whether a location is within a bounding box
    Args:
        latitude: latitude portion of location
        longitude: longitude portion of location
        replace_lat: replacement value for latitude
        replace_long: replacement value for longitude
        bounding_box: dictionary with max and min values to compare the location with
               
    Returns:
        latitude, longitude: 1 if out of range, 0 otherwise
    '''    
    if ((latitude > bounding_box['max_lat']) or (latitude < bounding_box['min_lat'])) \
    or ((longitude > bounding_box['max_long']) or (longitude < bounding_box['min_long'])):
        # replace both lat and long if either is outside of bounding box
        latitude = replace_lat
        longitude = replace_long  
        
            
    return(latitude, longitude)

In [ ]:
def replace_time(date_time_value,time_value):
    ''' given a datetime replace the time portion '''
     
    date_time_value = date_time_value.replace(hour=time_value.hour,minute=time_value.minute,second=time_value.minute)
    return(date_time_value)

In [ ]:
# master cell to call the other functions
# get the path for data files
path = "/content/drive/MyDrive/Colab Notebooks/NYC_PricePrediction/data/"
print("path is ",path)
config = get_config('/content/drive/MyDrive/Colab Notebooks/NYC_PricePrediction/notebooks/data_preparation_config.yml')
print("past config definition")
logging.getLogger().setLevel(logging.WARNING)
logging.warning("logging check")
print_config_values(config)
# load dataframe
df = ingest_data(path,config['file_names']['input_csv'],config['file_names']['pickled_input_dataframe'],config['general']['save_raw_dataframe'],config['general']['load_from_scratch'])
print("columns is "+str(config['columns']))
print("category_defaults is "+str(config['category_defaults']))
# fill missing values according to the defaults per column
df = fill_missing(df,config['columns'],config['category_defaults'])
# get assessment results after filling missing values
basic_assessment(df,config['columns'],config['valid_values'],config['non_negative_continuous'])
# df = replace_invalid_values(df,columns,valid_values,invalid_value_replacements,non_neg_continuous)
df = replace_invalid_values(df,config['columns'],config['valid_values'],config['category_invalid_value_replacements'],config['non_negative_continuous'])
df = geo_replacement(df,config['latitude_replacement'],config['longitude_replacement'], config['bounding_box'])
geo_assessment(df,config['bounding_box'])
if config['general']['save_transformed_dataframe']:
    print("path is ",path)
    file_name = os.path.join(path,config['file_names']['pickled_output_dataframe'])
    print("file_name is ",file_name)
    df.to_pickle(file_name)
df.head()

path is  /content/drive/MyDrive/Colab Notebooks/NYC_PricePrediction/data/
path_to_yaml /content/drive/MyDrive/Colab Notebooks/NYC_PricePrediction/notebooks/data_preparation_config.yml
past config definition
config value  general   {'load_from_scratch': False, 'save_raw_dataframe': False, 'save_transformed_dataframe': True, 'remove_bad_values': True}
config value  columns   {'categorical': ['neighbourhood_group', 'neighbourhood', 'room_type'], 'continuous': ['minimum_nights', 'number_of_reviews', 'reviews_per_month', 'calculated_host_listings_count', 'latitude', 'longitude'], 'date': ['last_review'], 'text': ['name', 'host_name'], 'excluded': ['price', 'id']}
config value  category_defaults   {'categorical': 'missing', 'continuous': 0.0, 'text': 'missing', 'date': datetime.date(2019, 1, 1), 'excluded': 'missing'}
config value  category_invalid_value_replacements   {'categorical': 'bad_categorical', 'continuous': 'bad_continuous', 'text': 'bad_text', 'date': 'bad_date', 'exclude': 'bad_e

,id,name,host_id,host_name,neighbourhood_group,neighbourhood,latitude,longitude,room_type,price,minimum_nights,number_of_reviews,last_review,reviews_per_month,calculated_host_listings_count,availability_365,"(latitude, longitude)"
0,2539,Clean & quiet apt home by the park,2787,John,Brooklyn,Kensington,40.64749,-73.97237,Private room,149,1,9,2018-10-19,0.21,6,365,"(40.647490000000005, -73.97237)"
1,2595,Skylit Midtown Castle,2845,Jennifer,Manhattan,Midtown,40.75362,-73.98377,Entire home/apt,225,1,45,2019-05-21,0.38,2,355,"(40.75362, -73.98376999999999)"
2,3647,THE VILLAGE OF HARLEM....NEW YORK !,4632,Elisabeth,Manhattan,Harlem,40.80902,-73.94190,Private room,150,3,0,2019-01-01,0.00,1,365,"(40.809020000000004, -73.9419)"
3,3831,Cozy Entire Floor of Brownstone,4869,LisaRoxanne,Brooklyn,Clinton Hill,40.68514,-73.95976,Entire home/apt,89,1,270,2019-07-05,4.64,1,194,"(40.685140000000004, -73.95976)"
4,5022,Entire Apt: Spacious Studio/Loft by central park,7192,Laura,Manhattan,East Harlem,40.79851,-73.94399,Entire home/apt,80,10,9,2018-11-19,0.10,1,0,"(40.79851, -73.94399)"
